In [0]:
import re
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class HSCodes():
    def __init__(self, *paths):
        self.hs2_raw = pd.read_csv(paths[0],dtype={"hs2":object,"hs2 name":object})
        self.hs4_raw = pd.read_csv(paths[1],dtype={"hs4":object,"hs4 name":object})
        self.hs6_raw = pd.read_csv(paths[2],dtype={"hs6":object,"hs6 name":object})
        self.hs9_raw = pd.read_csv(paths[3],dtype={"hs9":object,"hs9 name":object})

        self.clean("underscore")

        self.hs2_merged = pd.DataFrame()
        self.hs4_merged= pd.DataFrame()
        self.hs6_merged= pd.DataFrame()
        self.hs9_merged= pd.DataFrame()

        self.hs_all = pd.DataFrame()

    def clean(self, method):
        if method == "underscore":
            for f in [(self.hs2_raw,"hs2"), (self.hs4_raw,"hs4"),\
                      (self.hs6_raw,"hs6"), (self.hs9_raw,"hs9")]:
                f[0][f[1]+"_name"] = f[0][f[1]+"_name"].str.replace("__",", ")
                f[0][f[1]+"_name"] = f[0][f[1]+"_name"].str.replace("_"," ")

    def mergeAndOrder(self):
        """Takes the descriptions of the HS codes at the four levels (2, 4, 6 nd 9)
        and creates a single files with descriptions merged in the correct
        hierarchical order. """
        hs_42 = self.hs4_raw.copy()
        hs_42["2_code"] = hs_42["hs4"].str[:2]
        hs42_merged = self.hs2_raw.merge(hs_42, left_on="hs2", right_on="2_code")
        hs_64 = self.hs6_raw.copy()
        hs_64["4_code"] = hs_64["hs6"].str[:4]
        hs642_merged = hs42_merged.merge(hs_64, left_on="hs4", right_on="4_code")
        hs_96 = self.hs9_raw.copy()
        hs_96["6_code"] = hs_96["hs9"].str[:6]
        hs9642_merged = hs642_merged.merge(hs_96, left_on="hs6", right_on="6_code")

        self.hs_all = hs9642_merged.copy()

        self.hs2_merged = self.hs2_raw.copy()
        self.hs4_merged["hs4"] = hs9642_merged["hs4"]
        self.hs4_merged["hs4_name"] =  hs9642_merged["hs2_name"] + " - " + hs9642_merged["hs4_name"]
        self.hs6_merged["hs6"] = hs9642_merged["hs6"]
        self.hs6_merged["hs6_name"] = hs9642_merged["hs2_name"] + " - " + hs9642_merged["hs4_name"]\
                                    + hs9642_merged["hs6_name"]
        self.hs9_merged["hs9"] = hs9642_merged["hs9"]
        self.hs9_merged["hs9_name"] = hs9642_merged["hs2_name"] + " - " + hs9642_merged["hs4_name"]\
                                    + hs9642_merged["hs6_name"] + " - " + hs9642_merged["hs9_name"]
        
        return self.hs2_merged.copy(), self.hs4_merged.copy(),\
                self.hs6_merged.copy(), self.hs9_merged.copy()


In [0]:
hs_2_path = "/content/drive/My Drive/Colab Notebooks/Data/hs2_eng.csv"
hs_4_path = "/content/drive/My Drive/Colab Notebooks/Data/hs4_eng.csv"
hs_6_path = "/content/drive/My Drive/Colab Notebooks/Data/hs6_eng.csv"
hs_9_path = "/content/drive/My Drive/Colab Notebooks/Data/hs9_eng.csv"

save_path = "/content/drive/My Drive/Colab Notebooks/Data/"

if __name__ == "__main__":
    rawCodes = HSCodes(hs_2_path, hs_4_path, hs_6_path, hs_9_path)
    hs2, hs4, hs6, hs9 = rawCodes.mergeAndOrder()
    for f in [(hs4, "hs4_fixed.csv"), (hs6, "hs6_fixed.csv"), (hs9, "hs9_fixed.csv")]:
        f[0].to_csv(save_path + f[1])


       hs4                                           hs4_name 2_code
0     0000                                             unknow     00
1     0101              Live horses, asses, mules and hinnies     01
2     0102                                Live bovine animals     01
3     0103                                         Live swine     01
4     0104                               Live sheep and goats     01
...    ...                                                ...    ...
1219  9702        Original engravings, prints and lithographs     97
1220  9703  Original sculptures and statuary, in any material     97
1221  9704  Postage or revenue stamps, stamp-postmarks, fi...     97
1222  9705  Collections and collectors' pieces of zoologic...     97
1223  9706     Antiques of an age exceeding one hundred years     97

[1224 rows x 3 columns]
     hs2  ... 2_code
0     01  ...     01
1     01  ...     01
2     01  ...     01
3     01  ...     01
4     01  ...     01
...   ..  ...    ...
